In [16]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model
import os
from src.utils.metrics import obtener_ruta_app
from keras.losses import mean_squared_error

In [17]:
base_path = obtener_ruta_app("TemplateAgroIA")
model_path = os.path.join(base_path, "models", "modelo_RNN_Papas.h5")
scaler_path = os.path.join(base_path, "models", "scaler_RNN_Papas.pkl")

# === 1. Cargar modelo y scaler ===
model = load_model(model_path, custom_objects={'mse': mean_squared_error})
print("Modelo cargado.")

with open(scaler_path, 'rb') as f:
    scaler = pickle.load(f)
print("Scaler cargado.")

# === 2. Crear DataFrame con 3 meses de datos ===
df_prueba = pd.DataFrame({
    'area': [100, 110, 105],
    'PRECTOTCORR_SUM': [250, 260, 240],
    'RH2M': [80, 78, 82],
    'T2M': [22.1, 23.5, 21.7],
    'T2MDEW': [18.0, 17.5, 18.2],
    'T2MWET': [19.0, 19.5, 18.8],
    'T2M_MAX': [29, 30, 28],
    'T2M_MIN': [15, 16, 14],
    'WS2M': [1.2, 1.0, 1.3],
    'mes': [4, 5, 6],
    'produccion': [1200, 1250, 1230]
})
df_prueba['fecha'] = pd.date_range('2023-04-01', periods=3, freq='M')

columnas_features = ['area', 'PRECTOTCORR_SUM', 'RH2M', 'T2M', 'T2MDEW',
                     'T2MWET', 'T2M_MAX', 'T2M_MIN', 'WS2M', 'mes']

# === 3. Preparar input ===
def preparar_input(df, columnas_features, scaler, ventana_tiempo=3):
    df = df.sort_values("fecha")
    data = df[columnas_features + ['produccion']].values
    data_scaled = scaler.transform(data)
    X = data_scaled[-ventana_tiempo:, :-1]  # Solo features
    return np.expand_dims(X, axis=0)

# === 4. Predecir ===
def predecir(model, scaler, df_input, columnas_features):
    X = preparar_input(df_input, columnas_features, scaler)
    pred_scaled = model.predict(X, verbose=0)[0][0]

    # Desnormalizar solo la producción
    temp = np.zeros((1, scaler.n_features_in_))
    temp[0, -1] = pred_scaled
    pred_real = scaler.inverse_transform(temp)[0, -1]

    print(f"✅ Predicción del próximo mes: {pred_real:.2f}")
    return pred_real


Modelo cargado.
Scaler cargado.


C:\Users\Fiorella Victoria\AppData\Local\Temp\ipykernel_14796\3721225857.py:27: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_prueba['fecha'] = pd.date_range('2023-04-01', periods=3, freq='M')


In [18]:
predecir(model, scaler, df_prueba, columnas_features)

✅ Predicción del próximo mes: -5.94


np.float64(-5.936706787914037)